In [ ]:
"""
UNITS: UM=2.26 10**9, UR= 1 KPC, UV= 1 KM/S                    
G=1, UT=10 MYRS...
"""

#The variables included in the list are inputs in Francoise's code
name_par=['alam', 'rbar', 'cs', 'ai', 'beta=bincl', 'pa', 
          'phi', 'slit1', 'mm', 'ab', 'ac', 'ad',
          'ah', 'rd', 'mb', 'mc', 'md','ivbulb','rmax',
          'npr', 'npt','nx', 'vh', 'qtoom', 'hzero', 'idv2',
          'imodel', 'omb', 'ikal', 'phibar0', 'red2', 'red3',
          'istar', 'rilr1', 'rilr2', 'ndiv', 'ff'
         ]


def franc(xy_mesh, phibar0, ab, ac, ad, mb, mc, md, rd, phi, rmax): #ignore xy_mesh
    phi=np.deg2rad(phi)  

    rbar2=rbar*rbar
    cs2=cs*cs
    
    xcen=(nx+1.)/2.
    ycen=xcen

    #mm nbr of arms, ai pitch angle, omb pattern speed, ff spiral strength..
    ang=pa-np.pi/2.
    cang=np.cos(ang)
    sang=np.sin(ang)
     #
    """
    difference of convention: Shu phi=0 if bar //Oy
    then you can add p0
    """
    p0=-np.pi/2.
    p0=0.0
    """
    if p0=0, bar along Ox for phi=0
    also convention: phi rotates in the retro sense.....
    """

    ci=np.cos(ai)
    si=np.sin(ai)
    #   beta, pa, inclinaison, Pos Angle (do not confuse with beta)
    cb=np.cos(beta) 
    cb2=cb*cb
    sb=np.sin(beta) 
    sb2=sb*sb
    cp=np.cos(pa) 
    cp2=cp*cp
    sp=np.sin(pa) 
    sp2=sp*sp
    factor=si/mm  #
    #c--------------------------------------------------------------------------
    vunit=1.
    ut=10.
    uv=vunit
    uv2=uv*uv
    #print('vunit=',vunit,' km/s, ut=',ut,' Myr')
    g=0.971
    sq2=np.sqrt(2.)

    ab2=ab*ab  #
    ac2=ac*ac #
    ad2=ad*ad #
    ah2=ah*ah #
    vh2=vh*vh
    soft=0.002**2

    rd2=rd*rd
    rd3=rd2*rd
    v3=g*mb*(ab2+rd**2)**(1/2)/rd3 #*rd**2
    vc3=g*mc/(1-(1+rd**2/ac2)**(-1/2))
    vd3=g*md/(1-(1+rd**2/ad2)**(-1/2))
    amu0=vd3/(2*g*np.pi*ad**2)
    amu0=amu0+vc3/(2*g*np.pi*ac**2)

    sv3=np.sqrt(v3)
    svd3=np.sqrt(vd3)
    svc3=np.sqrt(vc3)

    SIGzd= vd3/(2*g*np.pi*ad2) #ad*md*(rd**2+ad**2)**(-3/2)/(2*np.pi)
    SIGzc= vc3/(2*g*np.pi*ac2) #ac*mc*(rd**2+ac**2)**(-3/2)/(2*np.pi)
    SIGzb= ivbulb*v3/(g*2*np.pi*ab2)
    SIGze= SIGzd+SIGzb+SIGzc

    #cc add bulge? ivbulb=1

    aniso=1.
    #calcul of rotation curve, etc..
    rmin=0
    dr=(rmax-rmin)/(npr-1)
    dt=2*np.pi/(npt-1)
    dx=2*rmax/(nx)
    dy=dx
    xmoy=0.
    ymoy=0.
    xmin=-rmax
    ymin=-rmax

    """
    C=====================================
    C  Loop on bar orientations!
    C=====================================
    """
    #phi=phi-np.deg2rad(5)
    #phi=phi+np.deg2rad(5)
    """
    c---------------------------------------------------------------------------
    ccc radial loop
    c---------------------------------------------------------------------------
    """
    iorb=0
    ipoint=0
    P2max=-1e8

    sigma=rad=vdisk=vbul=vdark=sigz=sigr=sigt=vrot=ome=cap=omm=omp=xspi=yspi=xspi2=yspi2=np.linspace(0,0,npr)
    xorb=yorb=vpol=disp2=vzer=np.zeros(shape=(npr,npt))
    vcir=vper=vtot=sigv2=np.zeros(shape=(nx,ny))
    #display(rad)
    for ir in range(1,npr): #do 1 ir=1,npr

        iorb=iorb+1
        rad[ir]=rmin+(ir-0.5)*dr
        #display(ir)
        rr=rad[ir]
        rr2=rr**2
        rt2=rr**2+soft
        rr3=rr2*rr
        amu=SIGzd*(1+rr**2/ad**2)**(-3/2)
        amu=amu+SIGzc*(1+rr2/ac2)**(-3/2)
        sigma[ir]=amu/amu0*200.
        #print(sigma[ir])
    #cc add bulge?
        amu=amu+ivbulb*v3/g/(2.*np.pi*ab2*(1.+ rr2/ab2)**(1.5))

        omeg=svd3*(rr2+ad2+soft)**(-0.75)
        vdisk[ir]=omeg*rr*uv
        capa2=4.*(ad2+.25*rr2)/(ad2+rr2)*omeg**2
    #ccc    add disque CND
        omegc=svc3/(rr2+ac2+soft)**(0.75)
        omeg=np.sqrt(omeg**2+omegc**2)     
    #ccc    add spheroide (bulge)...
        omeg3=sv3/(rr2+ab2+soft)**(0.75)
        vbul[ir]=rr*omeg3*uv
        omeg=np.sqrt(omeg**2+omeg3**2)
    #ccc    add spheroide (halo-dark matter)...
        omeg3h=vh*sq2/(rr2+ah2+soft)**(0.5)
        vdark[ir]=rr*omeg3h*uv
        omeg=np.sqrt(omeg**2+omeg3h**2)

        omeg2=omeg**2
        capa2b=4.*(ab2+0.25*rr2)/(ab2+rr2)*omeg3**2
        capa2=capa2b+capa2
        capa2c=4.*(ac2+0.25*rr2)/(ac2+rr2)*omegc**2
        capa2=capa2c+capa2
        capa2h=2.*(2.*ah2+rr2)/(ah2+rr2)*omeg3h**2
        capa2=capa2h+capa2
        capa=np.sqrt(capa2)
        OMEM=omeg-capa/2.
        OMEP=omeg+capa/2.
        DOME2= -3.*vd3*rr2*(ad2+rt2)**(-2.5)                               
        DOME2 = DOME2 -3.*vc3*rr2*(ac2+rt2)**(-2.5)
        DOME2 = DOME2 -3.*v3*rr2*(ab2+rt2)**(-2.5)
        DOME2 = DOME2 -4.*vh2*rr2*(ah2+rt2)**(-2)
    #c !!  d(ome2)/dr*rr in fact

    # calcul of dispersion, + asymmetric drift
        dcap2dr=4.*DOME2/rr - 3.*vd3*rr*((ad2+rt2)**(-2.5))*(2.-5.*rr2/(ad2+rt2))-\
                              3.*vc3*rr*((ac2+rt2)**(-2.5))*(2.-5.*rr2/(ac2+rt2))-\
                              3.*v3*rr*((ab2+rt2)**(-2.5))*(2.-5.*rr2/(ab2+rt2))-\
                              8.*vh2*rr/(ah2+rt2)**(5/2)*(1.-2*rr2/(ah2+rt2))
    #c is in fact dcap2/dr

        SIGMAr=vd3/g/(2.*np.pi*ad2*(1.+ rr2/ad2)**(1.5))                       

        SIGMAr=SIGMAr+vc3/g/(2.*np.pi*ac2*(1.+ rr2/ac2)**(1.5))                       

        SIGMAr=SIGMAr+ivbulb*v3/g/(2.*np.pi*ab2*(1.+ rr2/ab2)**(1.5)) 
    #ccc   add bulge?
        #qtoomr=qtoom/np.exp(0.8*rr2/rd2)                                     
        qtoomr=qtoom
        dmudr=-3.*rr/ad2*(1+rr2/ad2)**(-2.5)*SIGzd/SIGze

        dmudr=dmudr-3.*rr/ac2*(1+rr2/ac2)**(-2.5)*SIGzc/SIGze

        dmudr=dmudr-3.*rr/ab2*(1+rr2/ab2)**(-2.5)*SIGzb/SIGze
    #ccc   add bulge?

        if capa>=1e-5:      
            vcrit=(3.36*g*SIGMAr)/capa*qtoomr                         
            vtet=(capa/2./omeg)*vcrit/aniso

        else:       
            vcrit=0.     
            vtet=0.                     

    #ccc   calcul of loss of rotation, due to dispersion, dv2r is negatif

        vcrit2=vcrit*vcrit
        vtet2=vtet*vtet                  
        dv2r=rr/SIGMAr*SIGze*(3.36*g*qtoomr)**2*(3.*SIGMAr**2*dmudr/capa2-SIGMAr**3/SIGze*dcap2dr/capa2**2)+vcrit2-vtet2 

    #C   H=SIGZ2/(DEPI*G*MUR)=HZERO=INDEP DE R                               
    #C   SIGZ2 INDEP DE Z.... 
    #ccc plus generally sig_z**2 = F_z H
    #c       SIGZ2=2.*PI*HZERO*G*SIGMA   

        ome2=vd3*(rr2+ad2+soft)**(-1.5)
        capad2=4.*(ad2+.25*rr2+soft)/(ad2+rr2+soft)*ome2
        znu2=2*np.pi*g*SIGMAr/hzero 
        znu2=max(znu2,0.)
        sigz2=hzero**2*znu2
        sigz[ir]=uv*np.sqrt(sigz2)
        sigr[ir]=uv*vcrit
        sigt[ir]=uv*vtet

        if idv2==0:
            dv2r=0


        if idv2==2:
    #ccc autre expression de dv2= (BT87)
            dlogmu=-3.*rr2/ad2/(1+rr2/ad2)*SIGzd/SIGze
            dlogmu=dlogmu -3.*rr2/ac2/(1+rr2/ac2)*SIGzc/SIGze
            dv2r=-vtet2 +1.5*vcrit2 -0.5*sigz2 +2.*dlogmu*vcrit2 +0.5*(vcrit2-sigz2)

        blab=dv2r*uv2
        dv2r=min(dv2r,0.)
        vreal2=omeg2*rr2+dv2r
        vreal=np.sqrt(np.abs(vreal2))
    #C       write(6,278)omeg*rr*uv, vreal*uv,blab
    # 278	format(2x, 'vcir=', f8.2,' vreal=',f8.2,'dv2r=',e13.3)

    #cc  end dispersions

        vrot[ir]=rr*omeg*uv
        ome[ir]=omeg*uv
        cap[ir]=capa*uv
        omm[ir]=ome[ir]-cap[ir]/2.
        omp[ir]=ome[ir]+cap[ir]/2.
        tiny=1e-6
        if imodel==2:
            pom=2.*(omeg-omb)
            if abs(pom)<tiny: pom=np.sign(pom)*tiny #sign(tiny,pom)
            pom2=pom*pom

    #c------------------------------------
    #C simple models of bars
        if ikal==0:
            cphi=phibar0/rbar/rbar2
            phibar=phibar0/rbar2
            if rr<rbar:
                phibar=rr*phibar0*(rbar2-rr2)**(1.5) 
                cphi=-3*rr2*phibar/(rbar2-rr2) +phibar/rr
        else:
    #   Bar of Kalnajs................................ 
            cphi=0.
            phibar=0.
            qbar=phibar0
            XBAR=rr2/rbar2
            XBAR2=XBAR*XBAR
            XBAR3=XBAR*XBAR2
            BLABLA=1.06*qbar/(rr2*rbar2)
            if rr<rbar:
                POTRED=(1.-2.5*XBAR+2.1875*XBAR2-0.65625*XBAR3)
                phibar=+0.5*BLABLA*POTRED*rr2*rr2
                DPHIR=+BLABLA*rr2*(1.-5.*XBAR+6.5625*XBAR2-2.625*XBAR3)
                cphi=DPHIR*rr
            else:
    #C        COMPLETION OF EXTERNAL BAR....
                BLUB=-0.0641509433962264147*BLABLA/XBAR3
                DPHIR=BLUB*rr2
                cphi=DPHIR*rr
                phibar=-BLUB*rr2*rr2/4.

    #CC end choice Kalnajs or not
    #C---------------------------------------------
        dphi=2.*phibar/np.sqrt(rr2+soft)
    #cc dphi=2.*phibar/rr
    #cc  this is the  tang max force
        alam=alam*omeg
        alam2=alam*alam
        P2=dphi/omeg**2/rr
        ###print('rr=', rr, 'P2=', P2)
        P2max=max(P2max,P2)

        Aort=(4.*omeg2-capa2)/4./omeg
        denom=np.sqrt((capa2-pom2)**2+16.*pom2*alam2)
        blib=pom*cphi+2.*omeg*dphi
        aep= blib/pom/denom
        bep=(dphi-2*omeg*pom*aep)/pom2

        defas=math.atan2(-4.*pom*alam, capa2-pom2)
        #defas=math.atan2(-4.*pom*alam*blib+2*alam*cphi*(pom2-capa2),(capa2-pom2)*blib)

    # cc  better calculation
#         bep=-np.sqrt(dphi**2+4*aep**2*omeg2*pom2 - 4*aep*dphi*omeg*pom*np.cos(defas))/pom2
     #cc  better calcul with less approximations
#         bmax=rr/2
#         amax=abs((dphi-pom2**bmax)/2/omeg/pom)
#         if abs(bep)>bmax: bep=np.sign(bep)*abs(bmax)
#         if abs(aep)>amax : aep=np.sign(aep)*abs(amax)

    #cc epicycles run in the reverse sense

    # """
    # cc  better calcul
        #defas=math.atan2(-4.*pom*alam*blib+2*alam*cphi*(pom2-capa2),(capa2-pom2)*blib)
    # cc end better calcul
    # """
        defasb=math.atan2(dphi-2*omeg*pom*aep*np.cos(defas), 2*omeg*pom*aep*np.sin(defas))
    #   harm 2 (red2)
        harm=2
        harm2=harm*harm
        denom=np.sqrt((capa2-harm2*pom2)**2+16.*harm2*pom2*alam2)
        aep2=red2*(harm*pom*cphi+2.*omeg*dphi)/harm/pom/denom
        bep2=(dphi-2*omeg*harm*pom*aep2)/pom2/harm2
        defas2=math.atan2(-4.*harm*pom*alam, capa2-harm2*pom2)
        defasb2=math.atan2(dphi-2*omeg*harm*pom*aep*np.cos(defas2), 2*omeg*harm*pom*aep*np.sin(defas2))
    #   harm 3 (red3)------------------------------
        harm=3
        harm2=harm*harm
        denom=np.sqrt( (capa2-harm2*pom2)**2+ 16.*harm2*pom2*alam2)
        aep3= red3*(harm*pom*cphi+ 2.*omeg*dphi)/harm/pom/denom
        bep3=(dphi-2*omeg*harm*pom*aep3)/pom2/harm2
        defas3=math.atan2(-4.*harm*pom*alam,capa2-harm2*pom2)
        defasb3=math.atan2(dphi-2*omeg*harm*pom*aep*np.cos(defas3), 2*omeg*harm*pom*aep*np.sin(defas3))

        if istar==1:
            defas=0
            if rr>rilr1 and rr<=rilr2: defas=np.pi
            defas2=defas
            defas3=defas


        if iorb>=ndiv:
            iorb=0
            ipoint=ipoint+1



    #cc  end imodel=2

    #cc  imodel =1, Shu          

        ka=-mm*ci/si/rr
        #display(ka)


        gu=factor*rr*omeg
        gv=factor*rr*capa2/2./omeg
        anu=mm*(omb-omeg)/capa
        ax=cs2/2./gu/gv
        anu2=anu*anu
        bet=1./(1.-anu2+ax)
        alpha=anu*bet

    #c------------azimutal loop---------------
        ipot=0
        iteta=0
        for it in range(1,npt): #do 26 it=1,npt
            ipot=ipot+1
            teta=it*dt
            ct=np.cos(teta)
            st=np.sin(teta)
            eta=np.pi-ka*rr*np.log(rr)+mm*(teta+phi)
    #cc the minimum of potential is eta=0 (ki=pi)
            teta1=(ka*rr*np.log(rr)-np.pi)/mm -phi
            xspi[ir]=rr*np.cos(teta1)
            yspi[ir]=rr*np.sin(teta1)*cb
            #display(rr)
            veta=ff*np.sqrt(2.*gu*gv)*alpha*np.cos(eta)
            vxi=ff*gv*bet*np.sin(eta)
    #cc rotation of pitch angle, to pass in (rr,teta)
            vrr=ci*veta-si*vxi
            vteta=si*veta+ci*vxi

            if imodel==2:
    #ccc choice epicycle
                ptet=teta+phi+p0
                vrr=-aep*pom*np.sin(2.*(ptet)+defas) - aep2*2*pom*np.sin(4.*(ptet)+defas2) - aep3*3*pom*np.sin(6.*(ptet)+defas3) 
                vteta=(bep*pom+2.*Aort*aep)*np.sin(2.*(ptet)+defasb) + (bep2*2*pom+2.*Aort*aep2)*np.sin(4.*(ptet)+defasb2) + (bep3*3*pom+2.*Aort*aep3)*np.sin(6.*(ptet)+defasb3)  
                if ipot>=1:
                    ipot=0
                    iteta=iteta+1
                    radi=rr+aep*np.cos(2.*(ptet)+defas) + aep2*np.cos(4.*(ptet)+defas2) + aep3*np.cos(6.*(ptet)+defas3)
                    tetai=teta+bep*np.cos(2.*(ptet)+defasb) + bep2*np.cos(4.*(ptet)+defasb2) + bep3*np.cos(6.*(ptet)+defasb3)
                    xorb[ipoint,iteta]=radi*np.cos(tetai)
                    yorb[ipoint,iteta]=radi*np.sin(tetai)
    #CC end imodel=2

    #ccc  suppose line of  nodes horiz
            vpol[ir,it]=(vrr*st+vteta*ct)*uv*sb

    #ccc calcul of dispersions  projected
            disp2[ir,it]=(sigz2*cb2+sb2*(vcrit2*st*st + vtet2*ct*ct))*uv2

    #ccc same for velocities at zero order ..
            #vrr   =0.
            vteta=vreal

            #vteta=rr*(omeg-omb)
    #cc to be in the rotating  ref frame

            vzer[ir,it]=(vrr*st+vteta*ct)*uv*sb
        #  end azimutal loop
        rad[ir]=rr
        
    #   end  radial loop

    # Obtention of table in X,Y, with projection
    for ix in range(1,nx):#do 36 ix=1,nx
        for iy in range(1,ny): #36 iy=1,ny
            xx=(ix-xcen)*dx
            yy=(iy-ycen)*dy/cb

            rr2=xx**2+yy**2
            rr=np.sqrt(rr2)

            if rr>=rmax:
                vcir[ix,iy]=0.
                vper[ix,iy]=0.
                sigv2[ix,iy]=0.
            else:
                if rr!=0.:
                     teta=math.atan2(yy,xx)
                else:
                     teta=-np.pi/2

                if teta<=0.: teta=teta+2*np.pi

                ir=rr/dr#+1
                ir1=ir#+1
                it=teta/dt#+1
                it1=it#+1
                xr=1.+rr/dr-ir
                xt=1.+teta/dt-it
                xrm=1.-xr
                xtm=1.-xt
                if it1>npt: it1=it1-npt
                if ir1>npr: ir1=npr
                ir=int(ir)
                ir1=int(ir1)
                it=int(it)
                it1=int(it1)
                vcir[ix,iy]=vzer[ir,it]*xrm*xtm+vzer[ir1,it]*xr*xtm \
                           +vzer[ir,it1]*xrm*xt+vzer[ir1,it1]*xr*xt

                vper[ix,iy]=vpol[ir,it]*xrm*xtm+vpol[ir1,it]*xr*xtm \
                           +vpol[ir,it1]*xrm*xt+vpol[ir1,it1]*xr*xt

                sigv2[ix,iy]=disp2[ir,it]*xrm*xtm \
                            +disp2[ir1,it]*xr*xtm \
                            +disp2[ir,it1]*xrm*xt+disp2[ir1,it1]*xr*xt

    for ix in range(1,nx): #do 37 ix=1,nx
        for iy in range(1,ny): #do 37 iy=1,ny
            vtot[ix,iy]=vcir[ix,iy]+vper[ix,iy]

    #ccc ! vtot with line of nodes  horiz, taking into account  beta...
    #ccc d or ang=pa-pi/2.

    ombv=omb*uv
    #ccc search Rcorotation
    blav=ome[1]-ombv
    for ir in range(2,npr): #do ir=2,npr
        bla=ome[ir]-ombv
        prod=blav*bla
        if prod<=0.: break
        blav=bla     

    rcor=rad[ir]
    
    return vtot